<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader

# Data loading and preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Base model (ResNet-18)
class ResNet18Mod(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18Mod, self).__init__()
        self.model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Define the search space and objective function
def search_space():
    return {
        'learning_rate': [1e-3, 1e-4, 1e-5],
        'optimizer': [optim.Adam, optim.SGD]
    }

def objective_function(hyperparams):
    model = ResNet18Mod()
    criterion = nn.CrossEntropyLoss()
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['learning_rate'])

    model.train()
    for epoch in range(3):  # Train for a few epochs
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    return evaluate(model)

def evaluate(model):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return correct / len(test_loader.dataset)

# Implementing a basic NAS loop
best_hyperparams = None
best_accuracy = 0

for learning_rate in search_space()['learning_rate']:
    for optimizer in search_space()['optimizer']:
        hyperparams = {'learning_rate': learning_rate, 'optimizer': optimizer}
        accuracy = objective_function(hyperparams)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparams = hyperparams

print(f"Best Hyperparameters: {best_hyperparams}")
print(f"Best Accuracy: {best_accuracy}")